# Load Data

In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)
import glob
orig_csv_path = r'/Users/dair/git/8696_DataAnalytics_Assignment/original_data/*.csv'
files = sorted(glob.glob(orig_csv_path))
frame = pd.DataFrame()
list_ = []
for file_ in files:
    df = pd.read_csv(file_, skiprows=7, encoding="latin1")
    list_.append(df)
frame = pd.concat(list_)
frame = frame.reset_index(drop=True)

# Clean Up Data

In [2]:
# Wind 'Calm' & blank = 0 | Made column numerical
frame.loc[frame['9am wind speed (km/h)'] == 'Calm', '9am wind speed (km/h)'] = 0
frame.loc[frame['3pm wind speed (km/h)'] == 'Calm', '3pm wind speed (km/h)'] = 0
frame.loc[frame['9am wind speed (km/h)'].isnull(), '9am wind speed (km/h)'] = 0
frame.loc[frame['3pm wind speed (km/h)'].isnull(), '3pm wind speed (km/h)'] = 0
# Wind Direction for calm entries (0 wind speed) = "CALM" | Ensured that there were categorical labels for all cells
frame.loc[frame['9am wind speed (km/h)'] == 0, '9am wind direction'] = 'CALM'
frame.loc[frame['3pm wind speed (km/h)'] == 0, '3pm wind direction'] = 'CALM'
# Blank rainfall cell (2018-03-28) = 0|No rainfall data available
frame.loc[frame["Date"] == '2018-03-28', 'Rainfall (mm)'] = 0
# Cloud Cover blank = 0 | There were no 0 values, assumed that blanks are 0)
frame.loc[frame['9am cloud amount (oktas)'].isnull(), '9am cloud amount (oktas)'] = 0
frame.loc[frame['3pm cloud amount (oktas)'].isnull(), '3pm cloud amount (oktas)'] = 0
# Min Temp for 2017-07-31T00:00:00Z) = 2 |Data was missing, found data from alternative source:
# https://www.accuweather.com/en/au/canberra/21921/month/21921?monyr=7/01/2017
frame.loc[frame['Date'] == '2017-07-31', 'Minimum temperature (°C)'] = 2
# Convert Maximum wind gust time to minutes from 00:00|Made column numerical
def time_to_minutes(time):
    if not isinstance(time, str):
        return None
    hrs, mins = time.split(':')
    return 60 * int(hrs) + int(mins)
frame['Time of maximum wind gust'] = frame['Time of maximum wind gust'].apply(time_to_minutes)

# Dropped empty leading column
frame.drop(df.columns[[0]], axis=1, inplace=True)
# Removed Empty Evaporation and Sunshine Hours Columns
frame.drop('Evaporation (mm)', axis=1, inplace=True)
frame.drop('Sunshine (hours)', axis=1, inplace=True)

# Rename Columns for Convenience

In [3]:
# Rename Columns
rename_map = {
    "9am Temperature (°C)": "Temperature (C) 9am",
    "9am relative humidity (%)": "Relative humidity (%) 9am",
    "9am cloud amount (oktas)": "Cloud amount (oktas) 9am",
    "9am wind direction": "Wind direction 9am",
    "9am wind speed (km/h)": "Wind speed (km/h) 9am",
    "9am MSL pressure (hPa)": "MSL pressure (hPa) 9am",
    "3pm Temperature (°C)": "Temperature (C) 3pm",
    "3pm relative humidity (%)": "Relative humidity (%) 3pm",
    "3pm cloud amount (oktas)": "Cloud amount (oktas) 3pm",
    "3pm wind direction": "Wind direction 3pm",
    "3pm wind speed (km/h)": "Wind speed (km/h) 3pm",
    "3pm MSL pressure (hPa)": "MSL pressure (hPa) 3pm",
    "Minimum temperature (°C)": "Minimum temperature (C)",
    "Maximum temperature (°C)": "Maximum temperature (C)",
    
}
df = frame.rename(columns=rename_map)

# Data Aggregation

In [4]:
# Dates aggregated into a 'Season' column to investigate trends in each season

def get_season(date):
    month = int(date.split('-')[1])
    if month in [9, 10, 11]:
        return 'SPRING'
    if month in [12, 1, 2]:
        return 'SUMMER'
    if month in [3, 4, 5]:
        return 'AUTUMN'
    if month in [6, 7, 8]:
        return 'WINTER'
    return 'ERROR'
df.loc[:, 'Season'] = df['Date'].apply(get_season)

# Feature Creation

In [5]:
# Create columns with differences between 9am and 3pm measures

df["Temperature delta"] =       (pd.to_numeric(df["Temperature (C) 9am"]) -       pd.to_numeric(df["Temperature (C) 3pm"])).round(decimals=2)
df["Relative humidity delta"] = (pd.to_numeric(df["Relative humidity (%) 9am"]) - pd.to_numeric(df["Relative humidity (%) 3pm"])).round(decimals=2)
df["Cloud amount delta"] =      (pd.to_numeric(df["Cloud amount (oktas) 9am"]) -  pd.to_numeric(df["Cloud amount (oktas) 3pm"])).round(decimals=2)
# df[" delta"] = df["Wind direction 3pm"] - df["Wind direction 9am"]
df["Wind speed delta"] =        (pd.to_numeric(df["Wind speed (km/h) 9am"]) -     pd.to_numeric(df["Wind speed (km/h) 3pm"])).round(decimals=2)
df["MSL pressure delta"] =      (pd.to_numeric(df["MSL pressure (hPa) 9am"]) -    pd.to_numeric(df["MSL pressure (hPa) 3pm"])).round(decimals=2)

df["Temperature range"] =       (pd.to_numeric(df["Maximum temperature (C)"]) -   pd.to_numeric(df["Minimum temperature (C)"])).round(decimals=2)

# Create Rainfall Tomorrow Target Columns

In [6]:
# Column 'Rainfall Tomorrow (mm)', which is the Rainfall column shifted up one cell|Useful for modelling correlations
df.loc[:, 'IGNORE Rainfall Tomorrow (mm)'] = df['Rainfall (mm)'].shift(-1)
# Derived 'RainTomorrow' column True/False from 'Rainfall Tomorrow (mm)'|Required for final class prediction
df.loc[:, 'TARGET Rain Tomorrow'] = df['IGNORE Rainfall Tomorrow (mm)'] > 0
# Drop last row - it has no 'Rain Tomorrow' Value
df = df[:-1]

In [7]:
df

,Date,Minimum temperature (C),Maximum temperature (C),Rainfall (mm),Direction of maximum wind gust,Speed of maximum wind gust (km/h),Time of maximum wind gust,Temperature (C) 9am,Relative humidity (%) 9am,Cloud amount (oktas) 9am,Wind direction 9am,Wind speed (km/h) 9am,MSL pressure (hPa) 9am,Temperature (C) 3pm,Relative humidity (%) 3pm,Cloud amount (oktas) 3pm,Wind direction 3pm,Wind speed (km/h) 3pm,MSL pressure (hPa) 3pm,Season,Temperature delta,Relative humidity delta,Cloud amount delta,Wind speed delta,MSL pressure delta,Temperature range,IGNORE Rainfall Tomorrow (mm),TARGET Rain Tomorrow
0,2017-05-1,1.5,19.9,0.0,NNW,48.0,762.0,8.5,90,2.0,CALM,0,1018.4,18.6,52.0,2.0,NW,24,1015.5,AUTUMN,-10.1,38.0,0.0,-24,2.9,18.4,0.0,False
1,2017-05-2,5.2,16.0,0.0,NW,37.0,910.0,10.9,80,4.0,CALM,0,1018.7,14.7,65.0,8.0,NW,24,1017.8,AUTUMN,-3.8,15.0,-4.0,-24,0.9,10.8,0.0,False
2,2017-05-3,0.6,16.7,0.0,SE,31.0,819.0,8.0,62,0.0,SSE,20,1029.3,14.3,43.0,7.0,ENE,13,1027.6,AUTUMN,-6.3,19.0,-7.0,7,1.7,16.1,0.0,False
3,2017-05-4,-0.1,17.6,0.0,ENE,24.0,1096.0,7.9,73,0.0,SSE,6,1032.8,16.2,46.0,3.0,NNE,11,1029.3,AUTUMN,-8.3,27.0,-3.0,-5,3.5,17.7,0.0,False
4,2017-05-5,2.3,19.1,0.0,NNW,30.0,874.0,9.4,92,6.0,CALM,0,1032.0,17.6,51.0,2.0,NNW,19,1027.7,AUTUMN,-8.2,41.0,4.0,-19,4.3,16.8,0.2,True
5,2017-05-6,1.8,17.3,0.2,NNW,43.0,799.0,7.9,98,7.0,SSW,7,1025.5,16.8,70.0,5.0,NW,26,1021.6,AUTUMN,-8.9,28.0,2.0,-19,3.9,15.5,0.0,False
6,2017-05-7,4.2,15.2,0.0,NW,39.0,185.0,10.4,45,0.0,WNW,17,1025.5,14.2,46.0,0.0,N,20,1024.3,AUTUMN,-3.8,-1.0,0.0,-3,1.2,11.0,0.0,False
7,2017-05-8,-4.4,15.2,0.0,S,31.0,1071.0,7.6,64,0.0,SSE,6,1029.4,13.7,45.0,4.0,ESE,13,1026.3,AUTUMN,-6.1,19.0,-4.0,-7,3.1,19.6,0.0,False
8,2017-05-9,-1.0,17.7,0.0,S,17.0,618.0,9.2,68,0.0,ESE,6,1028.5,16.9,33.0,0.0,SE,6,1024.4,AUTUMN,-7.7,35.0,0.0,0,4.1,18.7,0.0,False
9,2017-05-10,-2.0,18.5,0.0,SW,17.0,813.0,7.1,81,0.0,ESE,6,1026.8,17.2,39.0,0.0,NW,4,1022.8,AUTUMN,-10.1,42.0,0.0,2,4.0,20.5,0.0,False


# Create variations of Target Columns

In [13]:
# Create Column 'IGNORE Rain Tomorrow Intensity' for rainfall intensity NONE/LIGHT/MODERATE/HEAVY/VIOLENT
def get_bom_rainfall_intensity(rainfall):
    if rainfall > 50:
        return 'VIOLENT'
    elif rainfall > 6:
        return 'HEAVY'
    elif rainfall > 2:
        return 'MODERATE'
    elif rainfall > 0:
        return 'LIGHT'
    elif rainfall == 0:
        return 'NONE'
    else:
        raise NotImplementedError()
df.loc[:, 'IGNORE Rain Tomorrow Intensity'] = df['IGNORE Rainfall Tomorrow (mm)'].apply(get_bom_rainfall_intensity)

# Create Column 'IGNORE Light UP' for rainfall intensity True/False
def get_more_than_light_rain(rainfall):
    if rainfall > 0:
        return True
    return False
df.loc[:, 'IGNORE Light UP'] = df['IGNORE Rainfall Tomorrow (mm)'].apply(get_more_than_light_rain)

# Create Column 'IGNORE Moderate UP' for rainfall intensity True/False
def get_more_than_moderate_rain(rainfall):
    if rainfall > 2:
        return True
    return False
df.loc[:, 'IGNORE Moderate UP'] = df['IGNORE Rainfall Tomorrow (mm)'].apply(get_more_than_moderate_rain)

# Create Column 'IGNORE Heavy UP' for rainfall intensity True/False
def get_more_than_heavy_rain(rainfall):
    if rainfall > 6:
        return True
    return False
df.loc[:, 'IGNORE Heavy UP'] = df['IGNORE Rainfall Tomorrow (mm)'].apply(get_more_than_heavy_rain)

# Create Column 'IGNORE Violent UP' for rainfall intensity True/False
def get_more_than_violent_rain(rainfall):
    if rainfall > 50:
        return True
    return False
df.loc[:, 'IGNORE Violent UP'] = df['IGNORE Rainfall Tomorrow (mm)'].apply(get_more_than_violent_rain)

# Create Column WEIGHT with weights inversely proportional to the count of each class in the dataset
WEIGHT_RAIN = sum(df['TARGET Rain Tomorrow'] == False) / len(df) / sum(df['TARGET Rain Tomorrow'] == True)
WEIGHT_NORAIN = sum(df['TARGET Rain Tomorrow'] == True) / len(df) / sum(df['TARGET Rain Tomorrow'] == False)
def get_weight(rain):
    if rain:
        return WEIGHT_RAIN
    elif not rain:
        return WEIGHT_NORAIN
    else:
        raise NotImplementedError()
df.loc[:, 'WEIGHT'] = df['TARGET Rain Tomorrow'].apply(get_weight)

In [14]:
df

,Date,Minimum temperature (C),Maximum temperature (C),Rainfall (mm),Direction of maximum wind gust,Speed of maximum wind gust (km/h),Time of maximum wind gust,Temperature (C) 9am,Relative humidity (%) 9am,Cloud amount (oktas) 9am,Wind direction 9am,Wind speed (km/h) 9am,MSL pressure (hPa) 9am,Temperature (C) 3pm,Relative humidity (%) 3pm,Cloud amount (oktas) 3pm,Wind direction 3pm,Wind speed (km/h) 3pm,MSL pressure (hPa) 3pm,Season,Temperature delta,Relative humidity delta,Cloud amount delta,Wind speed delta,MSL pressure delta,Temperature range,IGNORE Rainfall Tomorrow (mm),TARGET Rain Tomorrow,IGNORE Rain Tomorrow Intensity,IGNORE Light UP,IGNORE Moderate UP,IGNORE Heavy UP,IGNORE Violent UP,WEIGHT
0,2017-05-1,1.5,19.9,0.0,NNW,48.0,762.0,8.5,90,2.0,CALM,0,1018.4,18.6,52.0,2.0,NW,24,1015.5,AUTUMN,-10.1,38.0,0.0,-24,2.9,18.4,0.0,False,NONE,False,False,False,False,0.000914
1,2017-05-2,5.2,16.0,0.0,NW,37.0,910.0,10.9,80,4.0,CALM,0,1018.7,14.7,65.0,8.0,NW,24,1017.8,AUTUMN,-3.8,15.0,-4.0,-24,0.9,10.8,0.0,False,NONE,False,False,False,False,0.000914
2,2017-05-3,0.6,16.7,0.0,SE,31.0,819.0,8.0,62,0.0,SSE,20,1029.3,14.3,43.0,7.0,ENE,13,1027.6,AUTUMN,-6.3,19.0,-7.0,7,1.7,16.1,0.0,False,NONE,False,False,False,False,0.000914
3,2017-05-4,-0.1,17.6,0.0,ENE,24.0,1096.0,7.9,73,0.0,SSE,6,1032.8,16.2,46.0,3.0,NNE,11,1029.3,AUTUMN,-8.3,27.0,-3.0,-5,3.5,17.7,0.0,False,NONE,False,False,False,False,0.000914
4,2017-05-5,2.3,19.1,0.0,NNW,30.0,874.0,9.4,92,6.0,CALM,0,1032.0,17.6,51.0,2.0,NNW,19,1027.7,AUTUMN,-8.2,41.0,4.0,-19,4.3,16.8,0.2,True,LIGHT,True,False,False,False,0.006234
5,2017-05-6,1.8,17.3,0.2,NNW,43.0,799.0,7.9,98,7.0,SSW,7,1025.5,16.8,70.0,5.0,NW,26,1021.6,AUTUMN,-8.9,28.0,2.0,-19,3.9,15.5,0.0,False,NONE,False,False,False,False,0.000914
6,2017-05-7,4.2,15.2,0.0,NW,39.0,185.0,10.4,45,0.0,WNW,17,1025.5,14.2,46.0,0.0,N,20,1024.3,AUTUMN,-3.8,-1.0,0.0,-3,1.2,11.0,0.0,False,NONE,False,False,False,False,0.000914
7,2017-05-8,-4.4,15.2,0.0,S,31.0,1071.0,7.6,64,0.0,SSE,6,1029.4,13.7,45.0,4.0,ESE,13,1026.3,AUTUMN,-6.1,19.0,-4.0,-7,3.1,19.6,0.0,False,NONE,False,False,False,False,0.000914
8,2017-05-9,-1.0,17.7,0.0,S,17.0,618.0,9.2,68,0.0,ESE,6,1028.5,16.9,33.0,0.0,SE,6,1024.4,AUTUMN,-7.7,35.0,0.0,0,4.1,18.7,0.0,False,NONE,False,False,False,False,0.000914
9,2017-05-10,-2.0,18.5,0.0,SW,17.0,813.0,7.1,81,0.0,ESE,6,1026.8,17.2,39.0,0.0,NW,4,1022.8,AUTUMN,-10.1,42.0,0.0,2,4.0,20.5,0.0,False,NONE,False,False,False,False,0.000914


In [15]:
# write out the out df to a csv
df.to_csv("./Canberra070351_refined.csv", index=False)

In [16]:
sum(df['WEIGHT'])

0.9999999999999993

In [17]:
# Write out Rain Tomorrow days CSV
raintomorrow_only = df[df['TARGET Rain Tomorrow'] == True]
raintomorrow_only.to_csv("./Canberra070351_refined-raintomorrow-only.csv", index=False)